In [2]:
import os
import dotenv
from langchain.chains.llm import LLMChain
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 创建模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# ChatMessageHistory的使用

1. 记忆存储

In [2]:
from langchain.memory import ChatMessageHistory

# 实例化
history = ChatMessageHistory()

# 添加相关的消息存储
history.add_user_message("你好")
history.add_ai_message("很不高兴认识你")

# 打印存储的消息
print(history.messages)

[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很不高兴认识你', additional_kwargs={}, response_metadata={})]


2. 对接大模型

In [3]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("你好")
history.add_ai_message("很不高兴认识你")
history.add_user_message("帮我计算一下：1 + 1 * 8 = ？")

response = llm.invoke(history.messages)

print(response.content)

根据运算的优先级，乘法要先于加法。所以计算过程是：

1 + (1 * 8) = 1 + 8 = 9

所以，1 + 1 * 8 = 9。


## ConversationBufferMemory（基础）

1. 返回存储的字符串信息

In [1]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

# inputs对应的就是用户消息，outputs对应的就是AI消息
memory.save_context(inputs={"input": "你好"}, outputs={"output": "很不高兴认识你"})
memory.save_context(inputs={"input": "请帮我计算一下8 * 8 - 1 = ？"}, outputs={"output": "63"})

# 返回的字典结构的Key = history
print(memory.load_memory_variables({}))

{'history': 'Human: 你好\nAI: 很不高兴认识你\nHuman: 请帮我计算一下8 * 8 - 1 = ？\nAI: 63'}


/var/folders/bq/hh4nchrn1sdcx7xtqspm_2nc0000gn/T/ipykernel_87150/4171666384.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


2. 以消息列表的形式返回存储消息

In [3]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

# inputs对应的就是用户消息，outputs对应的就是AI消息
memory.save_context(inputs={"input": "你好"}, outputs={"output": "很不高兴认识你"})
memory.save_context(inputs={"input": "请帮我计算一下8 * 8 - 1 = ？"}, outputs={"output": "63"})

# 返回的字典结构的Key = history
print(memory.load_memory_variables({}))

print("\n")

# 原始消息列表
print(memory.chat_memory.messages)

{'history': 'Human: 你好\nAI: 很不高兴认识你\nHuman: 请帮我计算一下8 * 8 - 1 = ？\nAI: 63'}


[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很不高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='请帮我计算一下8 * 8 - 1 = ？', additional_kwargs={}, response_metadata={}), AIMessage(content='63', additional_kwargs={}, response_metadata={})]


3. 结合大模型，提示词模版的使用（PromptTemplate）

In [10]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains.llm import LLMChain

llm = ChatOpenAI(model="gpt-4o-mini")
prompt = PromptTemplate.from_template(
    template="""
    你可以和人类对话
    当前对话历史：{history}
    人类问题：{question}
    回复：
    """
)

memory = ConversationBufferMemory()
# memory = ConversationBufferMemory(memory_key="chat_history") #显式设置history的名称，修改之后使用chat_history代替history

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

response = chain.invoke({"question": "你好，我叫小红"})

print(response)

{'question': '你好，我叫小红', 'history': '', 'text': '你好，小红！很高兴和你聊天。你今天过得怎么样？'}


In [11]:

response = chain.invoke({"question": "你好，我是谁？"})

print(response)

{'question': '你好，我是谁？', 'history': 'Human: 你好，我叫小红\nAI: 你好，小红！很高兴和你聊天。你今天过得怎么样？', 'text': '你好，小红！你刚刚告诉我你叫小红。有什么我可以帮助你的吗？'}


4. 显式设置

5. 使用ChatPromptTemplate和return_messages

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-4o-mini")

# 创建Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个与人类对话的机器人"),
    MessagesPlaceholder("history"),
    ("human", "问题：{question}")
])

# 创建Memory
memory = ConversationBufferMemory(return_messages=True)

# 创建LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

res1 = llm_chain.invoke({"question": "中国首都在哪里？"})

print(res1, end="\n\n")

{'question': '中国首都在哪里？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={})], 'text': '中国的首都是北京。'}



In [16]:
res2 = llm_chain.invoke({"question": "我刚才问的什么问题？"})
print(res2)

{'question': '我刚才问的什么问题？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚才问的什么问题？', additional_kwargs={}, response_metadata={}), AIMessage(content='你刚才问的问题是“中国首都在哪里？”', additional_kwargs={}, response_metadata={})], 'text': '你刚才问的问题是“中国首都在哪里？”'}
